## Assignment 2
For this assignment you'll be looking at 2017 data on immunizations from the CDC.

Your datafile for this assignment is in assets/NISPUF17.csv.

A data users guide for this, which you'll need to map the variables in the data to the questions being asked, is available at assets/NIS-PUF17-DUG.pdf.

## Question 1
Write a function called `proportion_of_education` which returns the proportion of children in the
dataset who had a mother with the education levels equal to less than high school (<12), high school (12),
more than high school but not a college graduate (>12) and college degree.

*This function should return a dictionary in the form of (use the correct numbers, do not round numbers):*
```
    {"less than high school":0.2,
    "high school":0.4,
    "more than high school but not college":0.2,
    "college":0.2}
```

In [23]:
import pandas as pd

ds = pd.read_csv('assets/NISPUF17.csv', index_col=0)
ds = ds['EDUC1']
rows = ds.count()
counts = pd.DataFrame(ds.value_counts())
counts['Proportion'] = counts['EDUC1'] / rows
counts


,EDUC1,Proportion
4,13656,0.479747
3,6999,0.245881
2,4906,0.172352
1,2904,0.102020


In [24]:
def proportion_of_education():
    ds = pd.read_csv('assets/NISPUF17.csv', index_col=0)
    ds = ds['EDUC1']
    rows = ds.count()
    counts = pd.DataFrame(ds.value_counts())
    counts['Proportion'] = counts['EDUC1'] / rows

    return {
        'less than high school': counts.loc[1]['Proportion'],
        'high school': counts.loc[2]['Proportion'],
        'more than high school but not college': counts.loc[3]['Proportion'],
        'college': counts.loc[4]['Proportion']
    }

In [22]:
proportion_of_education()

{'less than high school': 0.10202002459160373,
 'high school': 0.172352011241876,
 'more than high school but not college': 0.24588090637625154,
 'college': 0.47974705779026877}

In [25]:
assert type(proportion_of_education())==type({}), "You must return a dictionary."
assert len(proportion_of_education()) == 4, "You have not returned a dictionary with four items in it."
assert "less than high school" in proportion_of_education().keys(), "You have not returned a dictionary with the correct keys."
assert "high school" in proportion_of_education().keys(), "You have not returned a dictionary with the correct keys."
assert "more than high school but not college" in proportion_of_education().keys(), "You have not returned a dictionary with the correct keys."
assert "college" in proportion_of_education().keys(), "You have not returned a dictionary with the correct keys."

## Question 2

Let's explore the relationship between being fed breastmilk as a child and getting a seasonal influenza
 vaccine from a healthcare provider.

Return a tuple of the average number of influenza vaccines for those children we know received breastmilk
as a child and those who know did not.

*This function should return a tuple in the form (use the correct numbers:*
```
(2.5, 0.1)
```

In [48]:
ds = pd.read_csv('assets/NISPUF17.csv')

#CBF_01 - 1 = Yes, 2 = No
ds.rename(columns = {'CBF_01': 'Breastfeeding', 'P_NUMFLU': 'Vaccines'}, inplace=True)

# Filter out those who we do not know were breastfed or not
ds = ds[ds['Breastfeeding'] <= 2]

ds = ds[['Breastfeeding', 'Vaccines']]
ds = ds.dropna()

ds = ds.set_index('Breastfeeding').sort_index()
ds

,Vaccines
Breastfeeding,
1,0.0
1,0.0
1,2.0
1,2.0
1,2.0
...,...
2,4.0
2,2.0
2,0.0


In [62]:
avg = ds.groupby(level='Breastfeeding').mean()

In [64]:
avg.loc[1, 'Vaccines']

1.8799187420058687

In [67]:
def average_influenza_doses():
    df = pd.read_csv('assets/NISPUF17.csv')

    #CBF_01 - 1 = Yes, 2 = No
    df.rename(columns = {'CBF_01': 'Breastfeeding', 'P_NUMFLU': 'Vaccines'}, inplace=True)

    # Filter out those who we do not know were breastfed or not
    df = df[df['Breastfeeding'] <= 2]

    df = df[['Breastfeeding', 'Vaccines']]
    df = df.dropna()

    df = df.set_index('Breastfeeding').sort_index()
    avg = df.groupby(level='Breastfeeding').mean()

    return avg.loc[1, 'Vaccines'], avg.loc[2, 'Vaccines']

In [68]:
average_influenza_doses()

(1.8799187420058687, 1.5963945918878317)

In [ ]:
assert len(average_influenza_doses())==2, "Return two values in a tuple, the first for yes and the second for no."

## Question 3
Calculate the ratio of the **number of children who contracted chickenpox but were vaccinated
 against it** versus those who were **vaccinated but did not contract chicken pox.** Return results by sex.

*This function should return a dictionary in the form of (use the correct numbers):*
```
    {"male":0.2,
    "female":0.4}
```

Note: To aid in verification, the `chickenpox_by_sex()['female']` value the autograder is looking for
starts with the digits `0.0077`.

In [92]:
df = pd.read_csv('assets/NISPUF17.csv')

#CBF_01 - 1 = Yes, 2 = No
df.rename(columns = {'P_NUMVRC': 'Vaccines'}, inplace=True)
df = df[df['HAD_CPOX'] <= 2] # Filter out unknown chickenpox values
df = df[df['Vaccines'] > 0]
df['HAD_CPOX'] = df['HAD_CPOX'] == 1
df['Count'] = 1
df = df[['SEX', 'HAD_CPOX', 'Count']] # 1 = Male, 2 = Female
df = df.set_index(['SEX', 'HAD_CPOX']).sort_index()
df

Count
SEX HAD_CPOX       
1   False         1
    False         1
    False         1
    False         1
    False         1
...             ...
2   True          1
    True          1
    True          1
    True          1
    True          1

[13951 rows x 1 columns]

In [99]:
grp = df.groupby(['SEX','HAD_CPOX'])
count = grp.count()
count

Count
SEX HAD_CPOX       
1   False      7028
    True         68
2   False      6802
    True         53

In [111]:
mNone = count['Count'][1, False]
mGot = count['Count'][1, True]
mRatio = mGot / (mNone + mGot)
fNone = count['Count'][2, False]
fGot = count['Count'][2, True]
fRatio = fGot / (fNone + fGot)
ratio = {'male': mRatio, 'female': fRatio}
ratio

{'male': 0.009582863585118376, 'female': 0.007731582786287381}

In [112]:
def chickenpox_by_sex():
    # Load
    df = pd.read_csv('assets/NISPUF17.csv')
    df.rename(columns = {'P_NUMVRC': 'Vaccines'}, inplace=True)

    # Filter
    df = df[df['HAD_CPOX'] <= 2] # Filter out unknown chickenpox values
    df = df[df['Vaccines'] > 0]

    # Transform Columns
    df['HAD_CPOX'] = df['HAD_CPOX'] == 1
    df['Count'] = 1

    # Form our data
    df = df[['SEX', 'HAD_CPOX', 'Count']] # 1 = Male, 2 = Female
    df = df.set_index(['SEX', 'HAD_CPOX']).sort_index()

    # Collapse down to grouped counts
    grp = df.groupby(['SEX','HAD_CPOX'])
    count = grp.count()

    # Male Ratio
    mNone = count['Count'][1, False]
    mGot = count['Count'][1, True]
    mRatio = mGot / (mNone + mGot)

    # Female Ratio
    fNone = count['Count'][2, False]
    fGot = count['Count'][2, True]
    fRatio = fGot / (fNone + fGot)

    return {'male': mRatio, 'female': fRatio}

In [113]:
assert len(chickenpox_by_sex())==2, "Return a dictionary with two items, the first for males and the second for females."

## Question 4
A correlation is a statistical relationship between two variables. If we wanted to know if vaccines work, we
might look at the correlation between the use of the vaccine and whether it results in prevention of the
infection or disease [1]. In this question, you are to see if there is a correlation between having had the
chicken pox and the number of chickenpox vaccine doses given (varicella).

Some notes on interpreting the answer. The `had_chickenpox_column` is either `1` (for yes) or `2` (for no),
and the `num_chickenpox_vaccine_column` is the number of doses a child has been given of the varicella
vaccine. A positive correlation (e.g., `corr > 0`) means that an increase in `had_chickenpox_column`
(which means more no’s) would also increase the values of `num_chickenpox_vaccine_column` (which means more
 doses of vaccine). If there is a negative correlation (e.g., `corr < 0`), it indicates that having had
 chickenpox is related to an increase in the number of vaccine doses.

Also, `pval` is the probability that we observe a correlation between `had_chickenpox_column` and
`num_chickenpox_vaccine_column` which is greater than or equal to a particular value occurred by chance.
A small `pval` means that the observed correlation is highly unlikely to occur by chance. In this case,
 `pval` should be very small (will end in `e-18` indicating a very small number).

[1] This isn’t really the full picture, since we are not looking at when the dose was given. It’s possible
that children had chickenpox and then their parents went to get them the vaccine. Does this dataset have the
 data we would need to investigate the timing of the dose?

In [16]:
import pandas as pd

# Load
df = pd.read_csv('assets/NISPUF17.csv')
df.rename(columns = {'P_NUMVRC': 'Vaccines'}, inplace=True)

# Filter
df = df[df['HAD_CPOX'] <= 2] # Filter out unknown chickenpox values

# Set Relevant Columns
df = df[['HAD_CPOX', 'Vaccines']]
df = df.dropna()

# Engineer Columns
def calcHadCPox(row):
    hadCpox = row['HAD_CPOX']
    if hadCpox == 1:
        row['CPOX_COUNT'] = 1
    else:
        row['CPOX_COUNT'] = 0
    return row
df = df.apply(calcHadCPox, axis='columns')

df = df[['Vaccines', 'CPOX_COUNT']]
df

,Vaccines,CPOX_COUNT
3,1.0,0.0
4,0.0,0.0
6,1.0,0.0
7,0.0,0.0
8,0.0,1.0
...,...,...
28446,1.0,0.0
28447,1.0,0.0
28449,1.0,0.0
28452,1.0,0.0


In [19]:
total = df.sum()
vaccines = total['Vaccines']
cpoxCount = total['CPOX_COUNT']

Vaccines      14166.0
CPOX_COUNT      173.0
dtype: float64

In [25]:
def corr_chickenpox():
    import scipy.stats as stats
    import numpy as np
    import pandas as pd

    # Load
    df = pd.read_csv('assets/NISPUF17.csv')
    df.rename(columns = {'P_NUMVRC': 'Vaccines'}, inplace=True)

    # Filter
    df = df[df['HAD_CPOX'] <= 2] # Filter out unknown chickenpox values

    # Set Relevant Columns
    df = df[['HAD_CPOX', 'Vaccines']]
    df = df.dropna()

    # Engineer Columns
    def calcHadCPox(row):
        hadCpox = row['HAD_CPOX']
        if hadCpox == 1:
            row['CPOX_COUNT'] = 1
        else:
            row['CPOX_COUNT'] = 0
        return row
    df = df.apply(calcHadCPox, axis='columns')

    # here is some stub code to actually run the correlation
    corr, pval=stats.pearsonr(df["CPOX_COUNT"],df["Vaccines"])

    # just return the correlation
    return corr

In [26]:
corr_chickenpox()

-0.07044873460147978

In [27]:
assert -1<=corr_chickenpox()<=1, "You must return a float number between -1.0 and 1.0."
